# LRU Cache [medium]

Design a data structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement the **LRUCache** class:

- **LRUCache(int capacity)** Initialize the LRU cache with positive size capacity.
- **int get(int key)** Return the value of the key if the key exists, otherwise return -1.
- **void put(int key, int value)** Update the value of the key if the key exists. Otherwise, add the key-value pair to the cache. If the number of keys exceeds the capacity from this operation, evict the least recently used key.

The functions **get** and **put** must each run in **O(1)** average time complexity.

 

## Example 1:

Input

["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]

[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]

Output

[null, null, null, 1, null, -1, null, -1, 3, 4]

|Explanation:|
|---|
|LRUCache lRUCache = new LRUCache(2); |
|lRUCache.put(1, 1); // cache is {1=1} |
|lRUCache.put(2, 2); // cache is {1=1, 2=2} |
|lRUCache.get(1);    // return 1 |
|lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3} |
|lRUCache.get(2);    // returns -1 (not found) |
|lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3} |
|lRUCache.get(1);    // return -1 (not found) |
|lRUCache.get(3);    // return 3 |
|lRUCache.get(4);    // return 4 |
 

## Constraints:

- 1 <= capacity <= 3000
- 0 <= key <= 104
- 0 <= value <= 105
- At most 2 * 105 calls will be made to **get** and **put**.

In [1]:
# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)
class LRUNode:
    def __init__(self, key:str="", val:int=0, prev: "LRUNode"=None, next:"LRUNode"=None):
        self.key = key
        self.value = val
        self.prev = prev
        self.next = next

class LRUCache:
    
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}
        self.count = 0

        # Create the head and tail dummy nodes and link them
        self.lru_node_head = LRUNode() # dummy node for head access
        self.lru_node_tail = LRUNode() # dummy node for tail access
        self.lru_node_head.next = self.lru_node_tail
        self.lru_node_tail.prev = self.lru_node_head

    # Must be O(1) time complexity
    def get(self, key: str) -> int:
        if key not in self.cache:
            return -1
        
        node = self.cache[key]
        self._move_to_head(node)
        return node.value
        
    # Must be O(1) time complexity
    def put(self, key: str, value: int) -> None:
        # Update value if key already exists
        if key in self.cache:
            node = self.cache[key]
            self._move_to_head(node)
            node.value = value

        # Else add a new node to the LRU
        else:
            node = LRUNode(key, value)
            self.cache[key] = node
            self._add_to_head(node)

            # Drop last node if we are over capacity
            if self.count > self.capacity:
                node = self._remove_tail()
                del self.cache[node.key]
        
    def _add_to_head(self, node:LRUNode) -> None:
        """Adds node to the head"""
        head = self.lru_node_head
        node.prev = head
        node.next = head.next
        head.next.prev = node
        head.next = node
        self.count += 1

    def _remove_node(self, node:LRUNode) -> None:
        """Removes a node"""
        node.prev.next = node.next
        node.next.prev = node.prev
        self.count -= 1

    def _move_to_head(self, node:LRUNode) -> None:
        """Moves a node to the head"""
        self._remove_node(node)
        self._add_to_head(node)

    def _remove_tail(self) -> LRUNode:
        """Removes the last node"""
        node = self.lru_node_tail.prev
        self._remove_node(node)
        return node

    def __len__(self) -> int:
        return self.count



In [2]:
cache = LRUCache(5)
cache.put('mouse', 1)
cache.put('cow', 1)
cache.put('louse', 1)
cache.put('seacow', 1)
cache.put('house', 2)
cache.get('mouse')
cache.put('noose', 2)

assert len(cache) == 5
assert cache.get('mouse') == 1
assert cache.get('cow') == -1

In [3]:
# Test capacity of 1
cache_single = LRUCache(1)
cache_single.put('a', 1)
assert len(cache_single) == 1
cache_single.put('b', 2)  # This should evict 'a'
assert len(cache_single) == 1
assert cache_single.get('a') == -1
assert cache_single.get('b') == 2

# Test updating existing values
cache = LRUCache(2)
cache.put('x', 10)
cache.put('y', 20)
cache.put('x', 30)  # Update value of 'x', should not evict 'y'
assert len(cache) == 2
assert cache.get('x') == 30
assert cache.get('y') == 20

# Test accessing items affects LRU order
cache = LRUCache(3)
cache.put('a', 1)
cache.put('b', 2)
cache.put('c', 3)
cache.get('a')  # Accessing 'a' makes it most recently used
cache.put('d', 4)  # Should evict 'b', not 'a'
assert cache.get('b') == -1
assert cache.get('a') == 1
assert cache.get('c') == 3
assert cache.get('d') == 4

# Test multiple gets and puts
cache = LRUCache(2)
cache.put('key1', 1)
cache.put('key2', 2)
assert cache.get('key1') == 1
cache.put('key3', 3)  # Evicts key2
assert cache.get('key2') == -1
cache.put('key4', 4)  # Evicts key1
assert cache.get('key1') == -1
assert cache.get('key3') == 3
assert cache.get('key4') == 4
